In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Interactive Earthquake Map</title>


  <!-- Include Leaflet CSS -->
  <link rel="stylesheet" href="https://unpkg.com/leaflet/dist/leaflet.css"/>
  <style>
    #map {
      height: 600px; /* Ensure the map has a height */
    }
  </style>
</head>
<body>
  <h2>Interactive Earthquake Map</h2>
  <div id="map"></div> <!-- Map container -->

  <!-- Include D3.js and Leaflet JS libraries -->
  <script src="https://d3js.org/d3.v6.min.js"></script>
  <script src="https://unpkg.com/leaflet/dist/leaflet.js"></script>

  <script>
    // Initialize map and intial vie
    var map = L.map('map').setView([37.7749, -122.4194], 5);  // Centered on San Francisco

    // Add TileLayer to the map 
    L.tileLayer("https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png").addTo(map);

    // Get earthquake data using D3.js
    d3.json("https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson").then(function(data) {

      // Function to set the color based on earthquake depth
      function getColor(depth) {
        if (depth > 500) {
          return "#ff0000"; // Red for depths > 500 km
        } else if (depth > 300) {
          return "#ff4500"; // Orange-red for depths > 300 km
        } else if (depth > 100) {
          return "#ff8c00"; // Orange for depths > 100 km
        } else if (depth > 50) {
          return "#ffd700"; // Yellow for depths > 50 km
        } else if (depth > 20) {
          return "#adff2f"; // Yellow-green for depths > 20 km
        } else {
          return "#32cd32"; // Green for depths <= 20 km
        }
      }

      // Function to set size of markers based on magnitude
      function getSize(magnitude) {
        return magnitude * 4; // Scale size of markers 
      }

      // Function to create the popup content for each of the earthquakes
      function getPopupContent(feature) {
        return "<b>Location:</b> " + feature.properties.place + 
               "<br><b>Magnitude:</b> " + feature.properties.mag + 
               "<br><b>Depth:</b> " + feature.geometry.coordinates[2] + " km";
      }

      // Create a GeoJSON layer and add it to the map
      L.geoJSON(data, {
        pointToLayer: function(feature, latlng) {
          // Ensure the data point is correctly placed at the longitude and latitude
          var lon = feature.geometry.coordinates[0];  // Longitude
          var lat = feature.geometry.coordinates[1];  // Latitude

          // Create a circle marker at the correct location
          var marker = L.circleMarker([lat, lon], {  // Using [lat, lon] for accurate location
            radius: getSize(feature.properties.mag), // Size based on magnitude
            fillColor: getColor(feature.geometry.coordinates[2]), // Color based on depth
            color: "#000000", // Border color
            weight: 0.5, // Border width
            opacity: 1,
            fillOpacity: 0.7 // Fill opacity
          });

          // Bind a tooltip to each marker with the earthquake details
          marker.bindTooltip(
            "<b>Location:</b> " + feature.properties.place + 
            "<br><b>Magnitude:</b> " + feature.properties.mag + 
            "<br><b>Depth:</b> " + feature.geometry.coordinates[2] + " km",
            {
              permanent: false,  // Tooltip is visible only when mouse hovers
              direction: "top"   // Position the tooltip at the top of the marker
            }
          );

          // Bind a popup with more detailed information when clicked
          marker.bindPopup(getPopupContent(feature));

          return marker;
        }
      }).addTo(map);

      // Create a legend control
      var legend = L.control({position: 'bottomright'});

      // Add the depth color legend content
      legend.onAdd = function(map) {
        var div = L.DomUtil.create('div', 'info legend');
        var depths = [0, 20, 50, 100, 300, 500]; // Depth ranges
        var labels = [];

        // Loop through depth ranges and generate a label with colored square
        for (var i = 0; i < depths.length; i++) {
          div.innerHTML +=
            '<i style="background:' + getColor(depths[i] + 1) + '"></i> ' +
            depths[i] + (depths[i + 1] ? '&ndash;' + depths[i + 1] + ' km<br>' : '+ km');
        }
        return div;
      };

      // Add the legend to the map
      legend.addTo(map);
    }).catch(function(error) {
      console.error("Error loading GeoJSON data: ", error);
    });
  </script>
</body>
</html>

